In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

from sklearn.model_selection import cross_val_score
pd.set_option('display.max_columns', None)

# Preparación de Datos:

1. Lea el dataset limpio que se guardó en el TP1 (punto 2 de conclusiones)
2. Realice una codificación adecuada de variables categóricas si es necesario, utilizando técnicas como One-Hot Encoding según corresponda.
3. Separe el dataset en entrenamiento y prueba (70% entrenamiento, 30% prueba).
4. Normalice o estandarice los atributos numéricos para asegurar que todas las variables tengan la misma escala.

In [21]:
df = pd.read_csv('./weatherAUS_clean.csv')
df = df.dropna(subset=['RainTomorrow'])
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-07-02,Adelaide,12.7,15.8,0.8,1.4,7.8,SW,35.0,SSW,SW,13.0,15.0,75.0,52.0,1022.4,1022.6,NaN,NaN,13.7,15.5,0.0,0.0
1,2008-07-03,Adelaide,6.2,15.1,0.0,1.8,2.1,W,20.0,NNE,SW,2.0,11.0,81.0,56.0,1027.8,1026.5,NaN,NaN,9.3,13.9,0.0,0.0
2,2008-07-04,Adelaide,5.3,15.9,0.0,1.4,8.0,NNE,30.0,NNE,NE,6.0,13.0,71.0,46.0,1028.7,1025.6,NaN,NaN,10.2,15.3,0.0,0.0
4,2008-07-06,Adelaide,11.3,15.7,0.0,1.4,1.5,NNW,52.0,NNE,NNW,15.0,22.0,62.0,62.0,1019.5,1016.2,NaN,NaN,13.0,14.4,0.0,1.0
5,2008-07-07,Adelaide,7.6,11.2,16.2,4.6,1.1,WSW,46.0,WNW,SW,17.0,13.0,83.0,88.0,1015.9,1017.9,NaN,NaN,9.8,9.3,1.0,1.0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142146 entries, 0 to 145409
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142146 non-null  object 
 1   Location       142146 non-null  object 
 2   MinTemp        142146 non-null  float64
 3   MaxTemp        142146 non-null  float64
 4   Rainfall       142146 non-null  float64
 5   Evaporation    142146 non-null  float64
 6   Sunshine       142146 non-null  float64
 7   WindGustDir    142146 non-null  object 
 8   WindGustSpeed  142146 non-null  float64
 9   WindDir9am     142146 non-null  object 
 10  WindDir3pm     142146 non-null  object 
 11  WindSpeed9am   142146 non-null  float64
 12  WindSpeed3pm   142146 non-null  float64
 13  Humidity9am    142146 non-null  float64
 14  Humidity3pm    142146 non-null  float64
 15  Pressure9am    142146 non-null  float64
 16  Pressure3pm    142146 non-null  float64
 17  Cloud9am       108123 non-nul

In [17]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
count,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,142146.000000,108123.000000,108123.000000,142146.000000,142146.000000,142146.000000,142146.000000
mean,12.187692,23.226294,2.361354,6.401686,7.052296,39.954716,14.005417,18.828388,68.874298,51.366729,1017.792458,1015.475242,4.772176,4.756518,16.962762,21.696234,0.224220,0.224199
std,6.404064,7.117627,8.479277,9.447599,3.641605,14.472044,8.898212,9.054297,19.248263,20.913647,6.946859,6.869402,2.833918,2.556619,6.511215,6.960287,0.417069,0.417055
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,0.000000,0.000000
25%,7.600000,17.900000,0.000000,1.800000,3.700000,30.000000,7.000000,13.000000,57.000000,36.000000,1013.500000,1011.000000,2.000000,2.000000,12.200000,16.600000,0.000000,0.000000
50%,12.000000,22.600000,0.000000,3.800000,7.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.200000,1015.500000,6.000000,5.000000,16.700000,21.100000,0.000000,0.000000
75%,16.800000,28.200000,0.800000,6.800000,10.500000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.400000,1019.900000,7.000000,7.000000,21.500000,26.400000,0.000000,0.000000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,1.000000,1.000000


Se observa que para los campos Cloud9am y Cloud3pm hay filas queno tienen información.

In [29]:
# Como a priori creo que la nubosidad debe ser un factor importante y vimos que hay filas nulas para los campos Cloud9am y Cloud3pm, hago la limpieza correspondiente
df_clean = df.dropna()
#descarto la columna Date, ya que no creo que las precipitaciones dependan de la fecha, sino de las condiciones que se dan en cada una de esas fechas
df_clean = df_clean.drop(columns=['Date'])
# Codificación de variables categóricas. En este caso son las que corresponden a la dirección del viento en las columnas 'Location','WindGustDir','WindDir9am','WindDir3pm'
df_encoded = pd.get_dummies(df_clean, columns=['Location','WindGustDir','WindDir9am','WindDir3pm'], dtype=float)
df_encoded

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Location_Albany,Location_Albury,Location_AliceSprings,Location_Ballarat,Location_Bendigo,Location_Brisbane,Location_Cairns,Location_Canberra,Location_Cobar,Location_CoffsHarbour,Location_Darwin,Location_Hobart,Location_Katherine,Location_Launceston,Location_Melbourne,Location_MelbourneAirport,Location_Mildura,Location_Moree,Location_MountGambier,Location_Newcastle,Location_NorfolkIsland,Location_Nuriootpa,Location_PearceRAAF,Location_Perth,Location_PerthAirport,Location_Portland,Location_Richmond,Location_Sale,Location_Sydney,Location_SydneyAirport,Location_Townsville,Location_Uluru,Location_WaggaWagga,Location_Watsonia,Location_Williamtown,Location_Wollongong,Location_Woomera,WindGustDir_E,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_E,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir3pm_E,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
3192,14.7,21.0,0.0,5.4,9.1,17.0,6.0,17.0,64.0,71.0,1012.8,1009.3,2.0,6.0,17.8,19.1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3193,16.5,24.0,0.2,4.8,10.0,17.0,19.0,24.0,36.0,73.0,999.8,998.7,7.0,7.0,20.0,21.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3194,14.1,20.6,7.8,7.0,7.2,17.0,13.0,41.0,69.0,81.0,1006.7,1008.5,5.0,7.0,19.8,18.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3195,13.0,17.8,0.0,3.6,4.9,17.0,13.0,19.0,73.0,63.0,1015.3,1016.3,3.0,5.0,16.0,17.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3196,14.8,18.5,0.0,3.8,8.7,17.0,11.0,20.0,46.0,48.0,1020.1,1017.9,8.0,3.0,16.9,17.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [42]:
# Separo nuestras variables predictoras de la variable a predecir
X = df_encoded.drop(columns=['RainTomorrow'])
y = df_encoded['RainTomorrow']

# Divido los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Normalizo las variables que no se encuentran entre los valores 0 y 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Miro algunos de los nuevos valores
print(X_train_scaled[:5])
print(X_test_scaled[:5])
print(y_train.head())
print(y_test.head())

[[-1.36596222 -1.28634907 -0.26576957 -0.28617063  0.86204515  0.80204575
   1.29634943  1.72941224 -0.14742118 -0.77489083  0.55390885  0.49604049
  -0.97684541 -0.68528216 -1.49667203 -1.21530724 -0.52597215 -0.16872509
  -0.17168394 -0.16847643 -0.1700869  -0.17000464 -0.17467864 -0.16864224
  -0.1806535  -0.16988118 -0.16685216 -0.17407529 -0.17237598 -0.11957971
  -0.16797815 -0.1489161  -0.16872509 -0.16889067  6.02055207 -0.16918011
  -0.16609772 -0.16905612 -0.16913879 -0.16197818 -0.17391408 -0.16831049
  -0.16718649 -0.16810284 -0.16864224 -0.17920563 -0.16913879 -0.17209132
  -0.12054228 -0.16951035 -0.16889067 -0.15630477 -0.16951035 -0.16843496
  -0.27916514 -0.25411723 -0.23882528 -0.262096   -0.23534374 -0.21079524
  -0.22709949 -0.23521865 -0.25190018 -0.28291435 -0.24672845 -0.2497279
  -0.2568462  -0.33064516 -0.23836132  3.21433289 -0.28302328 -0.25576312
  -0.25620262 -0.3018198  -0.25187052 -0.24624593 -0.24927974 -0.27169692
  -0.25444117 -0.28606172 -0.26871308 -

# Modelado y Evaluación:

1. Determine dos métricas de evaluación que considere importante medir para este problema. Justifique su elección.
2. Implemente un modelo de base usando como predicción que determine que llueve mañana si hoy llueve, y si hoy no llueve mañana no va a llover.
3. Implemente y entrene un modelo de k Nearest Neighbors (kNN) para predecir si va a llover mañana. Utilice la distancia euclidiana como medida de distancia entre vecinos.Ajuste el parámetro k utilizando técnicas como la validación cruzada (5 folds) para optimizar el rendimiento del modelo (utilice como función de optimización una de las métricas definidas en el punto anterior).
4. Implemente y entrene un modelo de regresión logística para predecir la misma variable objetivo.
5. Evalúe los dos modelos con las métricas definidas al principio. Discuta las fortalezas y debilidades de cada enfoque en el contexto específico de este problema comparándolos con el modelo de base.

1. Se utilizarán Precision y Recall:

Justificación:

- Precision: es útil para asegurarse que las predicciones positivas del modelo sean correctas.

- Recall: sirve para asegurar la identificación de todos los casos positivos.

2. Implementación del modelo de base:

In [73]:
y_pred_base = X_test['RainToday']

modeloBase = pd.DataFrame({
    'RainToday': X_test['RainToday'],   # Valor real
    'RainTomorrow': y_pred_base         # Predicción del modelo base
})

modeloBase.head()

,RainToday,RainTomorrow
141586,1.0,1.0
26510,0.0,0.0
27276,0.0,0.0
126350,1.0,1.0
30185,0.0,0.0


3. Modelo de k Nearest Neighbors (kNN):

In [74]:
# Implementar y entrenar el modelo kNN
knn = KNeighborsClassifier(metric='euclidean')

# Validación cruzada para ajustar el parámetro k
k_values = range(1, 9)  # Probando k desde 1 hasta 9
cross_val_scores = [cross_val_score(KNeighborsClassifier(n_neighbors=k, metric='euclidean'), 
                                    X_train_scaled, y_train, cv=5, scoring='accuracy').mean() 
                    for k in k_values]

# Selección del mejor k
best_k = k_values[cross_val_scores.index(max(cross_val_scores))]
knn = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
knn.fit(X_train_scaled, y_train)

# Predicciones
y_pred_knn = knn.predict(X_test_scaled)

In [76]:
# Mejor k:
print(best_k)

7


In [83]:
# Evaluar el modelo
print('Matriz de confusión:')
print(confusion_matrix(y_test, y_pred_knn))
print('\n')
print('Reporte de clasificación:')
print(classification_report(y_test, y_pred_knn))
print('\n')
print(f'Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}')

Matriz de confusión:
[[24125  1219]
 [ 4791  2302]]


Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.83      0.95      0.89     25344
         1.0       0.65      0.32      0.43      7093

    accuracy                           0.81     32437
   macro avg       0.74      0.64      0.66     32437
weighted avg       0.79      0.81      0.79     32437



Accuracy: 0.81


4. Modelo de Regresión Logística

In [55]:
# Entreno el modelo de regresión logística
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

LogisticRegression()

In [66]:
# Hacer predicciones
y_pred = model.predict(X_test_scaled)

# Evaluar el modelo
print('Matriz de confusión:')
print(confusion_matrix(y_test, y_pred))
print('\n')
print('Reporte de clasificación:')
print(classification_report(y_test, y_pred))
print('\n')
print(f'Accuracy: {accuracy_score(y_test, y_pred):.2f}')

Matriz de confusión:
[[23980  1364]
 [ 3524  3569]]


Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.87      0.95      0.91     25344
         1.0       0.72      0.50      0.59      7093

    accuracy                           0.85     32437
   macro avg       0.80      0.72      0.75     32437
weighted avg       0.84      0.85      0.84     32437



Accuracy: 0.85


In [82]:
from sklearn.metrics import precision_score, recall_score

# Evaluación del modelo de base
precision_base = precision_score(y_test, y_pred_base)
recall_base = recall_score(y_test, y_pred_base)

# Evaluación del modelo kNN
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)

# Evaluación del modelo de regresión logística
precision_logreg = precision_score(y_test, y_pred)
recall_logreg = recall_score(y_test, y_pred)

# Imprimir los resultados
print("Modelo Base - Precisión:", precision_base)
print("Modelo Base - Recall:", recall_base)
print("\nModelo KNN (k={}):".format(best_k))
print("Precisión:", precision_knn)
print("Recall:", recall_knn)
print("\nModelo Regresión Logística:")
print("Precisión:", precision_logreg)
print("Recall:", recall_logreg)

# Comparación de reportes de clasificación
print("\nReporte de clasificación Modelo Base:\n", classification_report(y_test, y_pred_base))
print("\nReporte de clasificación Modelo KNN:\n", classification_report(y_test, y_pred_knn))
print("\nReporte de clasificación del modelo de regresión logística:\n", classification_report(y_test, y_pred))

Modelo Base - Precisión: 0.45984383714445065
Modelo Base - Recall: 0.46496545890314395

Modelo KNN (k=7):
Precisión: 0.6537915364953139
Recall: 0.3245453263781193

Modelo Regresión Logística:
Precisión: 0.7234948307318062
Recall: 0.503172141548005

Reporte de clasificación Modelo Base:
               precision    recall  f1-score   support

         0.0       0.85      0.85      0.85     25344
         1.0       0.46      0.46      0.46      7093

    accuracy                           0.76     32437
   macro avg       0.65      0.66      0.66     32437
weighted avg       0.76      0.76      0.76     32437


Reporte de clasificación Modelo KNN:
               precision    recall  f1-score   support

         0.0       0.83      0.95      0.89     25344
         1.0       0.65      0.32      0.43      7093

    accuracy                           0.81     32437
   macro avg       0.74      0.64      0.66     32437
weighted avg       0.79      0.81      0.79     32437


Reporte de clasifi

# Conclusiones:

En base a los fundamentos que se expondrán a continuación, el mejor modelo es el de Regresión Logística.

Se procede a interpretar los resultados para cada modelo en términos de precisión (precision) y sensibilidad (recall). Si bien se hace foco en los resultados de la clase "llueve mañana", también se analiza el reporte de clasificoón completo, es decir que también se analiza la clase "no llueve mañana".

1. Modelo Base (si hoy llueve mañana también y si hoy no llueve mañana tampoco)

- Precisión: 0.46
- Recall: 0.46

    Con una precisión y un recall de 0.46, el modelo está acertando menos de la mitad de las veces cuando predice lluvia, y también está detectando menos de la mitad de los días en que realmente llueve. Este es un rendimiento bastante pobre y sirve como una línea base de comparación para modelos más sofisticados.

    Reporte de clasificación:

- Clase 0 (No llueve): Precisión de 0.85 y recall de 0.85, lo que indica que el modelo es bastante bueno para predecir los días en que no lloverá.

- Clase 1 (Lluvia): Precisión y recall de 0.46, lo que es bajo y muestra que el modelo base no es confiable para predecir días de lluvia.


2. Modelo k-Nearest Neighbors (kNN)

- Precisión: 0.65
- Recall: 0.32

    El modelo kNN muestra una precisión mejorada (0.65) en comparación con el modelo base, lo que significa que cuando predice que lloverá, es más probable que esté en lo correcto. Sin embargo, el recall ha bajado a 0.32, lo que indica que solo detecta aproximadamente un tercio de los días de lluvia.

    Reporte de clasificación:

- Clase 0 (No llueve): Aunque la precisión es menor que el modelo base (0.83), sigue siendo muy alta. Mejora significativa del recall (0.95), lo que indica que el modelo es excelente para predecir días en que no lloverá.
- Clase 1 (Lluvia): Aunque la precisión mejora a 0.65, el recall bajo (0.32) sugiere que este modelo tiene dificultades para identificar días de lluvia, lo que podría ser problemático si el objetivo es no perder ningún día de lluvia.

3. Modelo de Regresión Logística
- Precisión: 0.72
- Recall: 0.50

    El modelo de regresión logística ofrece la mejor precisión (0.72) entre los tres modelos, lo que indica que es más confiable para predecir lluvia. El recall también es mejor que en los modelos base y kNN, subiendo a 0.50 significa que está capturando la mitad de los días en los que realmente llueve.

    Reporte de clasificación:

- Clase 0 (No llueve): Alta precisión (0.87) y muy buen recall (0.95), lo que muestra que este modelo es fuerte en predecir correctamente los días sin lluvia.
- Clase 1 (Lluvia): Con una precisión de 0.72 y un recall de 0.50, este modelo es más equilibrado que kNN, aunque todavía pierde la mitad de los días de lluvia.

In [84]:
# Guardo los conjuntos de entrenamiento en CSV
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)

# Guardo los conjuntos de evaluación en CSV
X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

In [87]:
# Si bien en el TP3 usaré el modelo de regresión logística, decido guardar también el kNN

import pickle

# Guardo el modelo kNN en formato pickle
with open('modelo_kNN.pkl', 'wb') as file:
    pickle.dump(knn, file)

# Guardo el modelo de regresión logística en formato pickle
with open('modelo_regresionLogistica.pkl', 'wb') as file:
    pickle.dump(model, file)